# Problemas de trabajo práctico: Sesiones de doblaje
## Autor
Miguel Ángel Álvarez Cabanes
## Github
https://github.com/maalvarezcabanes/algoritmos_optimizacion

## Importación paquetes y funciones auxiliares

In [75]:
import pandas as pd
import random
import os

In [76]:
show_expanded_solution = True

## Enunciado
Se precisa coordinar el doblaje de una película. Las reglas de planificación que se deben seguir son:
1. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas.
2. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben.
3. No es posible grabar más de 6 tomas por día.
4. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible.

## Lectura de datos de tomas y actores
Los datos originales están en https://docs.google.com/spreadsheets/d/1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0/edit#gid=0, pero los descargo a un fichero "doblaje.xlsx" por comodidad.

A continuación leo los datos con pandas limpiando el DataFrame para solo quedarme con los datos de actores y tomas

In [77]:
df = pd.read_excel(os.path.join(".", "doblaje.xlsx"), skiprows=1).drop(["Unnamed: 11", "Total"], axis=1)
df.set_index("Toma", inplace=True)
df.drop("TOTAL", inplace=True)
df.dropna(inplace=True)
df

,1,2,3,4,5,6,7,8,9,10
Toma,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
6,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
7,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
8,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Planteamiento inicial: Solución naïve
Para la representación de datos voy a crear una lista de días, y cada uno de los días va a estar representado por otra lista de escenas que se ruedan ese día.

La solución naïve consistiría en elegir por orden en grupos de seis tomas por día, por lo que el máximo de días debería ser cinco. Voy a crear esta solución inicial ya que puede ser usada como una referencia de "peor solución".

Nota: Voy a tratar de hacer el código usando el menor número posible de funcionalidades de librerías para tratar de que la algoritmia usada quede lo más visible posible. Por comodidad sí usaré el DataFrame para acceder a la información.

In [97]:
def print_solucion(df, dias, coste, info_dias, show_expanded_solution = False):
    if show_expanded_solution:
        i = 0
        for dia, info in zip(dias, info_dias):
            print(f"Dia: {i}")
            print(f"Sesiones: {dia}")
            print(f"Uso actores: {info[0]}")
            print(f"Coste día: {info[1]}\n")
            i += 1
                                
    print(f"Solucion: {dias}")
    print(f"Coste total: {coste}")

In [98]:
def coste_dia(df, sesiones):
    coste = 0
    uso_actores = [sum(row) for index, row in df.loc[sesiones].T.iterrows()]
    for uso_actor in uso_actores:
        if uso_actor:
            coste += 1
    return (uso_actores, coste)

In [99]:
def coste_total(df, dias):
    coste = 0
    info_dias = []
    for num_dia, dia in enumerate(dias):
        (uso_actores, coste_aux) = coste_dia(df, dia)
        info_dias.append((uso_actores, coste_aux))
        coste += coste_aux
    return (info_dias, coste)

In [100]:
def solucion_naive(df, max_dias = 5, debug = False):
    sesiones = list(df.index)
    dias = []
    for i in range(max_dias):
        dias.append(sesiones[max_dias*i: max_dias*(i+1)])
    
    return (dias, coste_total(df, dias))

In [102]:
dias_naive, (info_dias_naive, coste_naive) = solucion_naive(df, debug=False)
print_solucion(df, dias_naive, coste_naive, info_dias_naive, show_expanded_solution = show_expanded_solution)

Dia: 0
Sesiones: [1, 2, 3, 4, 5]
Uso actores: [2.0, 4.0, 2.0, 3.0, 3.0, 0.0, 2.0, 2.0, 0.0, 0.0]
Coste día: 7

Dia: 1
Sesiones: [6, 7, 8, 9, 10]
Uso actores: [5.0, 5.0, 0.0, 3.0, 2.0, 2.0, 0.0, 0.0, 1.0, 0.0]
Coste día: 6

Dia: 2
Sesiones: [11, 12, 13, 14, 15]
Uso actores: [5.0, 3.0, 3.0, 2.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0]
Coste día: 8

Dia: 3
Sesiones: [16, 17, 18, 19, 20]
Uso actores: [3.0, 0.0, 4.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0]
Coste día: 6

Dia: 4
Sesiones: [21, 22, 23, 24, 25]
Uso actores: [3.0, 2.0, 3.0, 2.0, 0.0, 2.0, 0.0, 1.0, 0.0, 1.0]
Coste día: 7

Solucion: [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15], [16, 17, 18, 19, 20], [21, 22, 23, 24, 25]]
Coste total: 34


In [103]:
df2 = pd.DataFrame([["naïve", len(dias_naive), coste_naive]], columns = ["Algoritmo", "Numero dias", "Coste"])
df2

,Algoritmo,Numero dias,Coste
0,naïve,5,34


### Conclusiones
Como se puede observar, la solución requiere tres días de rodaje y un coste de 34 unidades.

## Aproximación voraz

In [104]:
def chequear_factibilidad(df, sesiones_dia, nueva_sesion, limite = 6, debug = False):
    uso_actores = [sum(row) for index, row in df.loc[sesiones_dia].T.iterrows()]
    posible_uso_actores = [sum(row) for row in zip(uso_actores, list(df.loc[nueva_sesion]))]
    if debug:
        print(f"Sesiones dia: {sesiones_dia}. Posible nueva sesion: {nueva_sesion}")
        print(f"Posible uso de actores: {posible_uso_actores}. Max {max(posible_uso_actores)}")
    if max(posible_uso_actores) > limite:
        return False
    else:
        return True

In [105]:
def solucion_voraz(df, sesiones, max_dias = 5, debug = False):
    dias = []
    to_remove = []
    for i in range(max_dias):
        if debug:
            print(f"Sesiones: {sesiones}")
            print(f"Usadas en iteracion anterior: {to_remove}")
        for z in to_remove:
            sesiones.remove(z)
        to_remove = []
        for j in sesiones:
            if len(dias) < i+1:
                dias.append([j])
                to_remove.append(j)
            else:
                if chequear_factibilidad(df, dias[i], j, debug=debug):
                    dias[i].append(j)
                    to_remove.append(j)
    
    if debug:
        print(f"Solucion: {dias}")
              
    return (dias, coste_total(df, dias))

In [106]:
sesiones = list(df.index)
dias_voraz, (info_dias_voraz, coste_voraz) = solucion_voraz(df, sesiones, debug=False)
print_solucion(df, dias_voraz, coste_voraz, info_dias_voraz, show_expanded_solution = show_expanded_solution)

Dia: 0
Sesiones: [1, 2, 3, 4, 5, 6, 7, 13, 14, 18, 21, 24]
Uso actores: [6.0, 6.0, 5.0, 6.0, 6.0, 4.0, 2.0, 3.0, 0.0, 0.0]
Coste día: 8

Dia: 1
Sesiones: [8, 9, 10, 11, 12, 15, 16, 27]
Uso actores: [6.0, 6.0, 2.0, 4.0, 2.0, 3.0, 1.0, 1.0, 1.0, 1.0]
Coste día: 10

Dia: 2
Sesiones: [17, 19, 20, 22, 23, 25]
Uso actores: [6.0, 2.0, 5.0, 3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]
Coste día: 6

Dia: 3
Sesiones: [26, 28, 29, 30]
Uso actores: [4.0, 0.0, 1.0, 2.0, 2.0, 1.0, 0.0, 0.0, 1.0, 0.0]
Coste día: 6

Solucion: [[1, 2, 3, 4, 5, 6, 7, 13, 14, 18, 21, 24], [8, 9, 10, 11, 12, 15, 16, 27], [17, 19, 20, 22, 23, 25], [26, 28, 29, 30]]
Coste total: 30


In [107]:
df2.loc[1] = ["voraz", len(dias_voraz), coste_voraz]
df2

,Algoritmo,Numero dias,Coste
0,naïve,5,34
1,voraz,4,30


## Búsqueda voraz multiarranque

In [109]:
def solucion_voraz_multiarranque(df, sesiones, intentos, max_dias = 5, debug = False):
    dias_voras_multi, coste_voraz_multi = dias_naive, coste_naive # :TODO: Chequear si pasarlo como parámetro
    for i in range(intentos):
        sesiones_aux = sesiones.copy()
        random.shuffle(sesiones_aux)
        if debug:
            print(f"Orden sesiones: {sesiones_aux}")
        dias_aux, (info_aux, coste_aux) = solucion_voraz(df, sesiones_aux, debug=debug)
        if debug:
            print(f"Coste voraz: {coste_aux}")
        if coste_aux < coste_voraz_multi:
            dias_voras_multi, coste_voraz_multi = dias_aux, coste_aux
            
    return (dias_voras_multi, coste_voraz_multi)

In [110]:
sesiones = list(df.index)
dias_voraz_multi, (info_dias_voraz_multi, coste_voraz_multi) = solucion_voraz_multiarranque(df, sesiones, 200, debug=False)
print_solucion(df, dias_voraz_multi, coste_voraz_multi, info_dias_voraz_multi, show_expanded_solution = show_expanded_solution)

TypeError: cannot unpack non-iterable int object

## Incorporando búsqueda local

In [ ]:
def busqueda_local(df, dias, coste):
    for i in dias